In [33]:
# df = pd.read_csv("../data/resultsdb/TierContests/tier-contests-2017.csv")
# df = df.drop(columns=['Unnamed: 0'])
# df = df[df.date != "date"]
# df.to_csv("../data/resultsdb/TierContests/tier-contests-2017.csv")

In [53]:
# df = pd.read_csv("../data/resultsdb/PlayerSalaries/player-salaries-2017.csv")
# df = df.drop(columns=['Unnamed: 0'])
# df = df[df.date != "date"]
# df.position.unique()
# df.to_csv("../data/resultsdb/PlayerSalaries/player-salaries-2017.csv")

In [108]:
df.to_csv("../data/resultsdb/TierContests/tier-contests-2017.csv")

In [1]:
import requests
import json
import pandas as pd
from IPython.core.debugger import set_trace ## put in the code to debug

# Date format MM/DD/YYYY
# Site DK=20 FD=2
DATE = "08/05/2019"
SITE = "20"
# Vars -- DATE, SITE
# This one can be used to get the player salaries
SLATES = "https://resultsdb-api.rotogrinders.com/api/slates?start=DATE&site=SITE"
# Vars -- SLATEID
CONTEST_OWNERSHIP = "https://resultsdb-api.rotogrinders.com/api/contest-ownership?_slateId=SLATEID"
# Vars -- CONTESTID, INDEXNUM
CONTEST_LINEUPS = "https://resultsdb-api.rotogrinders.com/api/entries?_contestId=CONTESTID&sortBy=points&order=desc&index=INDEXNUM&maxFinish=58750&players=&users=false&username=&isLive=false&minPoints=&maxSalaryUsed=&incomplete=false&positionsRemaining=&requiredStartingPositions="
# Vars SLATEID
CONTEST_SLATE = "https://resultsdb-api.rotogrinders.com/api/contests?slates=SLATEID&lean=true"
# Vars -- SLATEID (get summary)
SUMMARY = ""

# MLB Season Start and End Dates
start_date17 = date(2017, 4, 2)
end_date17 = date(2017, 10, 2)  # Ended Oct. 1

start_date18 = date(2018, 3, 29)
end_date18 = date(2018, 10, 2)  # Ended Oct. 1

start_date19 = date(2019, 3, 20)
end_date19 = date(2019, 9, 30)  # Ended Sept. 29

# NBA Season Start and End Dates
nba_start_date2018 = date(2018, 10, 15)  # nba 2018-19
nba_end_date2018 = date(2019, 4, 10)

In [2]:
"""
    pulls the different slates based on the date and the site
    date- date of the event 
    site- defaults to Draftkings=20, Fanduel=2
    sport- filters the slate by sport, defaults to None
    slate_type- fitlers on the type of contest, Classic by default
"""
sport_mapping = {"football": 1, "baseball": 2, "basketball": 3}
def get_slates(date, site="20", sport="baseball", slate_type="Classic"):
    slate = SLATES.replace("DATE", date).replace("SITE", site)
    slate_data = requests.get(slate).json()
    if sport is not None:
        slate_data = [slate for slate in slate_data if slate['sport'] == sport_mapping[sport]]
    if slate_type is not None:
        slate_data = [slate for slate in slate_data if slate['slateTypeName'] == slate_type]
    return slate_data

In [3]:
"""
    Fetches the ownerships for a particular slate. Returns the data as a dictionary of players 
    with individual contest ownership after
    slate_id - id of the slate found with the get_slates function
    returns daily_ownerships dict
"""
def get_contest_ownerships(slate_id):
    contest_ownership_url = CONTEST_OWNERSHIP.replace("SLATEID", slate_id)
    daily_ownerships = requests.get(contest_ownership_url).json()
    return daily_ownerships

In [4]:
"""
    Fetches the contests from the slate on that day.
    slate_id -- id of the slate to find contests for
    returns contests in that slate
"""
def get_contests_from_slate(slate_id):
    contests_in_slate_url = CONTEST_SLATE.replace("SLATEID", slate_id)
    contests_in_slate = requests.get(contests_in_slate_url).json()
    return contests_in_slate

In [195]:
def export_tiered_contest_lineups():
    pass

def export_contest_lineups(date):
    import itertools
    resultsdb_date = date.strftime("%m/%d/%Y")
    date = date.strftime("%Y-%m-%d")
    daily_slate = get_slates(resultsdb_date, SITE, sport="baseball")
    slate_dataframes = []
    for slate in daily_slate:
        slate_id = slate['_id']
        contests_in_slate = get_contests_from_slate(slate_id)
        tiers = get_tiers(contests_in_slate)
        for tier in range(0, len(tiers)):
            lineups_in_tier = get_lineups_in_tier(tiers[tier], date, tier)
            lineups_in_tier = list(itertools.chain.from_iterable(lineups_in_tier))
            columns = ['username', 'points', 'contest_rank',
                       'P1', 'P2', 'C', '1B', '2B', '3B', 'SS',
                       'OF1', 'OF2', 'OF3', 'date', 'contest_id', 'tier']
            df = pd.DataFrame(lineups_in_tier, columns=columns)
            df = df.drop_duplicates()
            slate_dataframes.append(df)
    return slate_dataframes

def get_lineups_in_tier(tier, date, tier_level):    
    lineups_in_tier = []
    for contest in tier:
        max_intervals = min(20, int(contest[1] / 25) + (contest[1] % 25 > 0))
        for interval in range(0, max_intervals):
            contest_lineups = get_contest_lineups(contest[0], interval)
            if len(contest_lineups) == 0:
                continue
            entries = contest_lineups['entries']
            created_rows = create_rows(entries, date, contest[0], tier_level)
            lineups_in_tier.append(created_rows)
    return lineups_in_tier

def create_rows(lineups, date, contest_id, tier):
    rows = []
    for i in range(0, len(lineups)):
        rows.append([lineups[i]['siteScreenName'], lineups[i]['points'], lineups[i]['rank']] +
                     parse_positions(lineups[i]['lineup']) + [date, contest_id, tier])
    return rows
        
def get_value(entry, pos, num):
    try:
        return entry[pos][num]['name']
    except:
        return ""

def parse_positions(entry):
    try:
        return [get_value(entry, "P", 0), get_value(entry, "P", 1), get_value(entry, "C", 0),
                get_value(entry, "1B", 0), get_value(entry, "2B", 0), get_value(entry, "3B", 0),
                get_value(entry, "SS", 0), get_value(entry, "OF", 0), get_value(entry, "OF", 1),
                get_value(entry, "OF", 2)]
    except IndexError:
        pass
    
# eg ../data/resultsdb/TierContestsBasketball/2019/{}-ownerships-{}.csv
def export_tiered_contest_ownerships(daily_slate, export_path):
    for slate in daily_slate:
        SLATEID = slate['_id']
        # skip slates where gameCount < 5
#         if slate.get('gameCount', 0) < 5:
#             continue
        # Get the names of the contests in the slate
        contests_in_slate = get_contests_from_slate(SLATEID)
        contest_names = [contest['name'] for contest in contests_in_slate]
        # Fetch daily ownerships
        daily_ownerships = get_contest_ownerships(SLATEID)
        # Parse data into a list to add to dataframe
        appended_player_data = []
        for player in daily_ownerships:
            player_data = daily_ownerships[player]
            player_row = [player_data.get('name', None), player_data.get('slatePosition', None),
                          player_data.get('salary', None), player_data.get('actualFpts', 0),
                          player_data.get('projectedFpts', 0), player_data.get('projectedOwnership', 0),
                          round(player_data.get('combinedOwnership', 0), 4)]
            player_contest_row = calculate_tiers(contests_in_slate, player_data)
            joined_list = player_row + player_contest_row + [date] + [SLATEID]
            appended_player_data.append(joined_list)
        # Create table structure from the data
        df = pd.DataFrame(appended_player_data, columns=['name',
         'slatePosition','salary', 'actualFpts', 'projectedFpts', 'projectedOwnership',
         'combinedOwnership', 'low_tier', 'mid_tier', 'high_tier', 'double_ups', 'date', 'slate_id'])
        df.to_csv(export_path.format(date, SLATEID))
        
def get_tiers(contests_in_slate, date):
    double_ups = filter((lambda x: "Double Up" in x['name']), contests_in_slate)
    double_ups = [(contest['_id'], contest.get('maxEntries', 0)) for contest in double_ups]
    low_tier = [(contest['_id'], contest.get('maxEntries', 0)) for contest in contests_in_slate if contest['entryFee'] <= 7]
    mid_tier = [(contest['_id'], contest.get('maxEntries', 0)) for contest in contests_in_slate if (contest['entryFee'] > 7 and contest['entryFee'] < 25)]
    high_tier = [(contest['_id'], contest.get('maxEntries', 0)) for contest in contests_in_slate if contest['entryFee'] >= 25]

    low_tier = [x for x in low_tier if x not in double_ups]
    mid_tier = [x for x in mid_tier if x not in double_ups]
    high_tier = [x for x in high_tier if x not in double_ups]
    return [double_ups, low_tier, mid_tier, high_tier, date]
    
def calculate_tiers(contests_in_slate, player_data):
    double_ups = filter((lambda x: "Double Up" in x['name']), contests_in_slate)
    double_ups = [contest['name'] for contest in double_ups]

    low_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] <= 7]
    mid_tier = [contest['name'] for contest in contests_in_slate if (contest['entryFee'] > 7 and contest['entryFee'] < 25)]
    high_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] >= 25]

    low_tier = [x for x in low_tier if x not in double_ups]
    mid_tier = [x for x in mid_tier if x not in double_ups]
    high_tier = [x for x in high_tier if x not in double_ups]
    
    low_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in low_tier]
    mid_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in mid_tier]
    high_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in high_tier]
    double_ups_ownerships = [player_data.get(contest_name, 0) for contest_name in double_ups]
    
    if len(low_tier_ownerships) != 0:
        average_low_tier = round(sum(low_tier_ownerships)/len(low_tier_ownerships), 4)
    else:
        average_low_tier = 0
    if len(mid_tier_ownerships) != 0:
        average_mid_tier = round(sum(mid_tier_ownerships)/len(mid_tier_ownerships), 4)
    else:
        average_mid_tier = 0
    if len(high_tier_ownerships) != 0:
        average_high_tier = round(sum(high_tier_ownerships)/len(high_tier_ownerships), 4)
    else:
        average_high_tier = 0
    if len(double_ups_ownerships) != 0:
        average_double_ups = round(sum(double_ups_ownerships)/len(double_ups_ownerships), 4)
    else:
        average_double_ups = 0
        
    return [average_low_tier, average_mid_tier, average_high_tier, average_double_ups]

In [9]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
# start_date = date(2017, 7, 16)
# end_date = date(2017, 10, 2)
# start_date = date(2018, 10, 15) #nba 2018-19
# end_date = date(2019, 4, 10)
# start_date = date(2019, 10, 1)
# end_date = date(2019, 11, 7)
start_date = date(2019, 4, 3)
end_date = date(2019, 10, 1)
from datetime import timedelta, date
from random import random

In [ ]:
tiers = ['double_ups', 'low_tier', 'mid_tier', 'high_tier']
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    result = export_contest_lineups(single_date)
    for res in range(0, len(result)):
        if len(result[res]) == 0:
            continue
        else:
            result[res].to_csv('../data/resultsdb/TierLineups/2019/{}-lineups-{}.csv'.format(date, random()))

In [ ]:
for single_date in daterange(start_date, end_date):
    resultsdb_date = single_date.strftime("%m/%d/%Y")
    date = single_date.strftime("%Y-%m-%d")
    daily_slate = get_slates(resultsdb_date, SITE, sport="basketball")
    export_tiered_contest_ownerships(daily_slate, export_path="../data/resultsdb/TierContestsBasketball/2019/{}-ownerships-{}.csv")

Lineups

Salaries

In [203]:
def parse_player_data(player, slate_id, date):
    return [player.get('team'), player.get('opponent'), player.get('projectedFpts'), player.get('projectedOwnership'),
           player.get('salary'), player.get('slatePosition'), player.get('dkName'), player.get('name'), slate_id, date]

def daterange(start_date, end_date):
    from datetime import timedelta, date
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
from datetime import timedelta, date
import itertools

# start_date = date(2017, 7, 25)
# end_date = date(2017, 10, 2)
# start_date = date(2018, 3, 29)
# end_date = date(2018, 10, 1)
start_date = date(2019, 3, 28)
end_date = date(2019, 10, 1)

columns = [
                "username",
                "points",
                "contest_rank",
                "P1",
                "P2",
                "C",
                "1B",
                "2B",
                "3B",
                "SS",
                "OF1",
                "OF2",
                "OF3",
                "date",
                "contest_id",
                "tier",
            ]

In [204]:
slates2017=[]
for single_date in daterange(start_date, end_date):
    get_date_slate = single_date.strftime("%m/%d/%Y")
    slates = get_slates(get_date_slate, sport="baseball")
    slate_tuples = [(get_date_slate, slate['_id']) for slate in slates]
    for tp in slate_tuples:
        slates2017.append(tp)

In [205]:
tiers_in_slate=[]
for slate in slates2017:
    contests = get_contests_from_slate(slate[1])
    tiers = get_tiers(contests, slate[0])
    tiers_in_slate.append(tiers)

In [208]:
for tiered_slate in tiers_in_slate:
    for tier in range(0,4):
        lit = get_lineups_in_tier(tiered_slate[tier], tiered_slate[4], tier)
        lineups_in_tier = list(itertools.chain.from_iterable(lit))
        df = pd.DataFrame(lineups_in_tier, columns=columns)
        df = df.drop_duplicates()
        path_date = tiered_slate[4].replace("/", "-")
        df.to_csv("../data/resultsdb/IndividualContest/2019/lineups-date-{}-tier-{}.csv".format(path_date, tier))

In [182]:
import copy
tiers_in_slate_duplicate = copy.deepcopy(tiers_in_slate)

In [207]:
start_date = date(2019, 3, 29)
end_date = date(2019, 7, 27)
for single_date in daterange(start_date, end_date):
    get_date_slate = single_date.strftime("%m/%d/%Y")
    tiers_in_slate = [tier for tier in tiers_in_slate if tier[4] != get_date_slate]

In [201]:
tiers_in_slate

[[[('5b0d42c618276d38f675eba2', 77),
   ('5b0d4281b3860538f19d57dd', 229),
   ('5b0c8d25b3860538f1953ad2', 1379),
   ('5b0c8d23b3860538f1953abb', 26),
   ('5b0c8d2418276d38f66c409c', 459),
   ('5b0d4212b3860538f19b8f8f', 1149)],
  [('5b0d424c18276d38f674e247', 5945),
   ('5b0d42af18276d38f675bcf6', 7134),
   ('5b0d426018276d38f6750266', 1189),
   ('5b0d424f18276d38f674e248', 1981)],
  [('5b0c8cd0b3860538f194f971', 17647),
   ('5b0d423518276d38f674a50c', 470),
   ('5b0d429618276d38f675773d', 1470),
   ('5b0d4263b3860538f19d3f64', 588)],
  [('5b0c8cd018276d38f66c3d23', 36),
   ('5b0c8cd0b3860538f194f972', 10),
   ('5b0d427bb3860538f19d5370', 185),
   ('5b0c8d2518276d38f66c4228', 1766),
   ('5b0c8cd018276d38f66c3d24', 925),
   ('5b0d42d5b3860538f19e382d', 227),
   ('5b0d423918276d38f674a681', 235),
   ('5b0c8d2418276d38f66c409d', 68),
   ('5b0d426818276d38f6750966', 10)],
  '05/28/2018'],
 [[('5b0d42c618276d38f675eba3', 111),
   ('5b0cc686b3860538f1998e8b', 3448),
   ('5b0d4281b3860538f19

Lines

In [190]:
def get_lineups_in_tier(tier, date, tier_level):    
    lineups_in_tier = []
    for contest in tier:
        max_intervals = min(int(contest[1] / 25) + int((contest[1] % 25 > 0)), 10)
        for interval in range(0, max_intervals):
            print(interval)
            contest_lineups = get_contest_lineups(contest[0], interval)
            if len(contest_lineups) == 0:
                continue
            entries = contest_lineups['entries']
            created_rows = create_rows(entries, date, contest[0], tier_level)
            lineups_in_tier.append(created_rows)
    return lineups_in_tier

In [175]:
"""
    Fetches the lineups 25 at a time based on the contest_id and index passed
    contest_id- id of the contest to fetch (found in the summary call)
    index- index of the lineups to fetch (fetches number 25*(index) - 25*(index+1))
    returns lineups
"""
def get_contest_lineups(contest_id, index=0):
    contest_lineup_url = CONTEST_LINEUPS.replace("CONTESTID", contest_id).replace("INDEXNUM", str(index))
    daily_lineups = requests.get(contest_lineup_url).json()
    return daily_lineups